## basic config

In [1]:

def align_faces_2(root_path):
    import os 
    current_path = os.getcwd()
    temp_path = "/home/bean/DragVideo/stylegan3-editing"
    os.chdir(temp_path)
    command =f"""python prepare_data/preparing_faces_parallel.py \
                --mode align \
                --root_path  {root_path} 
                """
    import subprocess
    subprocess.run(command, shell=True)
    os.chdir(current_path)

In [2]:

# ==== IMP ==================================================================
#  this import is must to avoid the error
# RuntimeError: GET was unable to find an engine to execute this computation
# import torch  # now not needed on .182
# ============================================================================

# # torch.cuda.is_available()
# print(torch.cuda.device_count())
# print(torch.__version__)


# why this is not working
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
# --------------------------------  configs   ------------------------------------------------

# DRAGVIDEO_ROOT_PATH = "/Ext_4T_SSD/ASHOK/"
DRAGVIDEO_ROOT_PATH = "/home/bean/"
EXPERIMENT_NAME_POSTFIX = "obama_60_frames"#"aligned_sg3_&_new_e4e_reconstruction_invstep_200_pti_step_450"

# keep temp paths here
#---------------------------------
sg3_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/stylegan3_3rdtime_weights/stylegan3-r-ffhqu-1024_module.pkl"
#defualt path ::  "stylegan2_ada_ffhq":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/ffhq.pkl",

# sg3_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/ffhq.pkl"
sg3_aligned = f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/stylegan3_3rdtime_weights/stylegan3-r-ffhq-1024_module.pkl"


new_e4e = f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/restyle_e4e_ffhq.pt"
old_e4e_with_pti = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/e4e_ffhq_encode.pt",

obama_video = "/home/bean/DragVideo/Data_store/OLD/original_videos/obama.mp4"

# --------------------------------  

import datetime
lazy_config = {
    "EXP_NAME": str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+EXPERIMENT_NAME_POSTFIX,
    "e4e": new_e4e,
    "stylegan2_ada_ffhq":  sg3_aligned,
    "video_path": obama_video,#  f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/person_speaking_original.mp4",
    "model_name" : "stylegan3", # stylegan2 or stylegan3
    "n_frames" : 60,
    "IMAGE_SIZE": 1024,
    "N_STEPS_in_draggan":  "100",
    
}

env_config = {
    "DragGAN_dir": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN",
    "Experiment_base_path":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/experiments/" ,
    "init_exp_dir_shell_path": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/init_datadirs.sh",
    "dummy_config_path" : f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy",
    
    
}

#hyper parameters in PTI
hyper_config = {
    "max_pti_steps": 450,
    "first_inv_steps": 200,
    "max_images_to_invert": 200,
}


# ----------------------------------------------------------------------------------------------

from run_utils import *

# create experiment data folder structure
Experiment_name = lazy_config["EXP_NAME"]
Experiment_base_path = env_config["Experiment_base_path"]
Experiment_path = os.path.join(Experiment_base_path, Experiment_name)

init_experiment_dir(Experiment_name,Experiment_base_path,shell_script_path=env_config["init_exp_dir_shell_path"])



# dummy paths_config overwrites the paths_config.py
# dummy_config_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy'
add_dummy_config_from_dict("hyperparameters.py", hyper_config,ROOT_PATH=env_config["dummy_config_path"])

# change path configs , hyperparameters 
paths_config_dict = {
    #pretrained models
    "e4e": lazy_config["e4e"],
    "stylegan2_ada_ffhq": lazy_config["stylegan2_ada_ffhq"],
    
    # to store tuned stylegan weights
    "checkpoints_dir": os.path.join(Experiment_path,'tuned_SG'),
    # to store latents
    "embedding_base_dir": os.path.join(Experiment_path,'latents'),
    # aligned / processed images
    "input_data_path": os.path.join(Experiment_path,'aligned'),
}

add_dummy_config_from_dict("paths_config.py", paths_config_dict,ROOT_PATH=env_config["dummy_config_path"])



# add all these configs to log.txt
# --------------------------------  
with open(os.path.join(Experiment_path,'log.txt'), 'a') as f:
    import json
    f.write(f"lazy_config: {json.dumps(lazy_config, indent=4)}\n")
    f.write(f"env_config: {json.dumps(env_config, indent=4)}\n")
    f.write(f"hyper_config: {json.dumps(hyper_config, indent=4)}\n")
    
    



In [3]:


from importlib.machinery import SourceFileLoader
# imports the module from the given path
video_utils = SourceFileLoader("video_utils","../utils_draggan/video_utils.py").load_module()
raw_path = os.path.join(Experiment_path, "raw")
from utils.align_data import pre_process_images

video_utils.extract_frames(lazy_config['video_path'], raw_path,n_frames=lazy_config['n_frames'])


 int(cam.get(cv2.CAP_PROP_FRAME_COUNT))= 119 ,n_frames=1 
Creating.../home/bean/DragVideo/Data_store/experiments/2023-08-16_02-00-05check_landmarks/raw/000.jpg


# start

In [4]:
# pre_process_images(raw_path, IMAGE_SIZE=lazy_config['IMAGE_SIZE']) # o/p: config.input_data_path 

align_faces_2(os.path.join(Experiment_path,'raw'))



1
Running on 1 paths
Here we goooo
	ForkPoolWorker-1 is starting to extract on #1 images
	Done!
Mischief managed in -1.2032074928283691s


In [5]:


from utils.align_data import pre_process_images
from scripts.run_pti import run_PTI

from run_utils_2 import load_generators,export_updated_pickle
from configs import paths_config

use_multi_id_training = True
model_id = run_PTI(use_wandb=False, use_multi_id_training=use_multi_id_training)


generator_type =paths_config.multi_id_model_type if use_multi_id_training else "__"
old_G, new_G = load_generators(model_id, generator_type)
sg_tuned_pkl = export_updated_pickle(new_G,model_id,name = lazy_config["model_name"])

print(sg_tuned_pkl) # 'QBUXQCXZGWET'



#get landmarks
landmark_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_facial-landmarks-recognition"
os.chdir(landmark_path)
print("pwd",os.getcwd())
import sys
sys.path.append(landmark_path)

from main import landmarks, dict_landmarks,show_landmarks,get_landmarks_dir

# to store landmarks
landmarks_dir =  os.path.join(Experiment_path,'landmarks')
processed_images_dir =  os.path.join(Experiment_path,'aligned')

# generate landmarks for all images in processed_images_dir
get_landmarks_dir(processed_images_dir,landmarks_dir)


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
current_directory:  /home/bean/DragVideo/DragGAN/_PTI
current_directory:  /home/bean/DragVideo/DragGAN/_PTI
torch.Size([3, 1024, 1024]) 3 1024 1024
Setting up PyTorch plugin "bias_act_plugin"... Done.


  0%|          | 0/200 [00:00<?, ?it/s]

Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %106 : int = prim::profile_ivalue(%104)
 does not have profile information (Triggered internally at /opt/conda/conda-bld/pytorch_1682343964576/work/third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  return forward_call(*args, **kwargs)
100%|██████████| 1/1 [00:00<00:00, 15.87it/s]


Exporting large updated pickle based off new generator and ffhq.pkl
/home/bean/DragVideo/Data_store/experiments/2023-08-16_02-00-05check_landmarks/tuned_SG/stylegan3_OTOUOCMYGNNX.pkl
pwd /home/bean/DragVideo/DragGAN/_facial-landmarks-recognition
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-08-16_02-00-05check_landmarks/landmarks/000.pkl
sg_tuned_pkl: /home/bean/DragVideo/Data_store/experiments/2023-08-16_02-00-05check_landmarks/tuned_SG/stylegan3_OTOUOCMYGNNX.pkl
intiating global state....
calling init_images......
Loading "/home/bean/DragVideo/Data_store/experiments/2023-08-16_02-00-05check_landmarks/tuned_SG/stylegan3_OTOUOCMYGNNX.pkl"... Done.
()
{'z_dim': 512, 'w_dim': 512, 'channel_base': 65536, 'channel_max': 1024, 'mapping_kwargs': {}, 'conv_kernel': 1, 'filter_size': 6, 'magnitude_ema_beta': 0.9988915792636801, 'output_scale': 0.25, 'c_dim': 0, 'img_resolution': 1024, 'img_channels': 3, 'use_radial_filters': True}
ws.shape: torch.Size([1, 16, 512]), self.num

/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343964576/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


ws.shape: torch.Size([1, 16, 512]), self.num_ws: 16, self.w_dim: 512
Current Source:
    [252, 498]
    [252, 568]
    [257, 638]
    [266, 710]
    [285, 783]
    [319, 851]
    [363, 912]
    [419, 961]
    [496, 975]
    [576, 959]
    [643, 915]
    [700, 858]
    [750, 794]
    [781, 723]
    [794, 646]
    [801, 573]
    [808, 497]
    [293, 464]
    [322, 426]
    [368, 407]
    [418, 406]
    [466, 421]
    [549, 420]
    [599, 405]
    [653, 407]
    [703, 427]
    [736, 464]
    [503, 468]
    [499, 514]
    [496, 559]
    [492, 608]
    [429, 634]
    [460, 644]
    [496, 655]
    [535, 641]
    [572, 630]
    [340, 494]
    [370, 488]
    [399, 485]
    [432, 489]
    [400, 492]
    [371, 495]
    [591, 486]
    [622, 483]
    [651, 486]
    [682, 492]
    [650, 491]
    [621, 489]
    [373, 732]
    [413, 699]
    [462, 688]
    [502, 693]
    [543, 685]
    [597, 692]
    [646, 722]
    [602, 783]
    [548, 815]
    [503, 822]
    [460, 819]
    [411, 792]
    [390, 734]


0

In [3]:
# clean all data from gpu
import torch
torch.cuda.empty_cache()

import os 
Experiment_path = "/home/bean/DragVideo/Data_store/experiments/2023-08-16_02-03-44obama_60_frames"
sg_tuned_pkl = "/home/bean/DragVideo/Data_store/experiments/2023-08-16_02-03-44obama_60_frames/tuned_SG/stylegan3_EGXIYKSTDCRR.pkl"



os.chdir(env_config["DragGAN_dir"])
import subprocess

print(f"sg_tuned_pkl: {sg_tuned_pkl}")

subprocess.call(['python', 'run_dragvideo.py', '--Experiment_path', Experiment_path, '--N_STEPS', lazy_config["N_STEPS_in_draggan"], '--CHECKPOINT_PATH', sg_tuned_pkl,"--MAX_SIZE",str(lazy_config["IMAGE_SIZE"]) ])


sg_tuned_pkl: /home/bean/DragVideo/Data_store/experiments/2023-08-16_02-03-44obama_60_frames/tuned_SG/stylegan3_EGXIYKSTDCRR.pkl
intiating global state....
calling init_images......
Loading "/home/bean/DragVideo/Data_store/experiments/2023-08-16_02-03-44obama_60_frames/tuned_SG/stylegan3_EGXIYKSTDCRR.pkl"... Done.
()
{'z_dim': 512, 'w_dim': 512, 'channel_base': 65536, 'channel_max': 1024, 'mapping_kwargs': {}, 'conv_kernel': 1, 'filter_size': 6, 'magnitude_ema_beta': 0.9988915792636801, 'output_scale': 0.25, 'c_dim': 0, 'img_resolution': 1024, 'img_channels': 3, 'use_radial_filters': True}
ws.shape: torch.Size([1, 16, 512]), self.num_ws: 16, self.w_dim: 512
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
ws.shape: torch.Size([1, 16, 512]), self.num_ws: 16, self.w_dim: 512
Running with:
    Source: [array([252, 498]), array([252, 568]), array([257, 638]), array([266, 710]), array([285, 783]), array([319, 851]), array([363, 912]), array([419, 961]), array([496, 975]), array([5

/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343964576/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


ws.shape: torch.Size([1, 16, 512]), self.num_ws: 16, self.w_dim: 512
Current Source:
    [252, 498]
    [252, 568]
    [257, 638]
    [266, 710]
    [285, 783]
    [319, 851]
    [363, 912]
    [419, 961]
    [496, 975]
    [576, 959]
    [643, 915]
    [700, 858]
    [750, 794]
    [781, 723]
    [794, 646]
    [801, 573]
    [808, 497]
    [293, 464]
    [322, 426]
    [368, 407]
    [418, 406]
    [466, 421]
    [549, 420]
    [599, 405]
    [653, 407]
    [703, 427]
    [736, 464]
    [503, 468]
    [499, 514]
    [496, 559]
    [492, 608]
    [429, 634]
    [460, 644]
    [496, 655]
    [535, 641]
    [572, 630]
    [340, 494]
    [370, 488]
    [399, 485]
    [432, 489]
    [400, 492]
    [371, 495]
    [591, 486]
    [622, 483]
    [651, 486]
    [682, 492]
    [650, 491]
    [621, 489]
    [373, 732]
    [413, 699]
    [462, 688]
    [502, 693]
    [543, 685]
    [597, 692]
    [646, 722]
    [602, 783]
    [548, 815]
    [503, 822]
    [460, 819]
    [411, 792]
    [390, 734]
